<a href="https://colab.research.google.com/github/pvarun75/nlp/blob/main/assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch


In [2]:
text = """
Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her.
"""


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load pre-trained tokenizer and model (for example, GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Encode the text
inputs = tokenizer(text, return_tensors="pt")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from transformers import AdamW

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop for different epochs
for epoch in [20, 60, 70]:
    print(f"Training for {epoch} epochs")
    for i in range(epoch):
        # Forward pass
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()

        # Optimizer step and zero grad
        optimizer.step()
        optimizer.zero_grad()

        # Print progress
        print(f"Epoch {i + 1}/{epoch}, Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training for 20 epochs
Epoch 1/20, Loss: 2.423823833465576
Epoch 2/20, Loss: 1.779335856437683
Epoch 3/20, Loss: 1.27019464969635
Epoch 4/20, Loss: 0.8682006597518921
Epoch 5/20, Loss: 0.5756881833076477
Epoch 6/20, Loss: 0.37283071875572205
Epoch 7/20, Loss: 0.19002920389175415
Epoch 8/20, Loss: 0.09569381177425385
Epoch 9/20, Loss: 0.07306008040904999
Epoch 10/20, Loss: 0.0532141849398613
Epoch 11/20, Loss: 0.03899887949228287
Epoch 12/20, Loss: 0.027239685878157616
Epoch 13/20, Loss: 0.017298053950071335
Epoch 14/20, Loss: 0.010061469860374928
Epoch 15/20, Loss: 0.005747366696596146
Epoch 16/20, Loss: 0.0034900037571787834
Epoch 17/20, Loss: 0.00233745900914073
Epoch 18/20, Loss: 0.0017175214597955346
Epoch 19/20, Loss: 0.0013531286967918277
Epoch 20/20, Loss: 0.001119360444135964
Training for 60 epochs
Epoch 1/60, Loss: 0.0009582411148585379
Epoch 2/60, Loss: 0.0008397672208957374
Epoch 3/60, Loss: 0.0007461330387741327
Epoch 4/60, Loss: 0.0006670918664894998
Epoch 5/60, Loss: 0.00

In [5]:
# Seed text
seed_text = "Once upon a time"

# Encode seed text
input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)

# Generate new text
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the


In [6]:
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_beams=5
)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her.

She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through
